# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
S. Shahaf  ->  S. Shahaf  |  ['S. Shahaf']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


J.-U. Pott  ->  J.-U. Pott  |  ['J.-U. Pott']
J.-U. Pott  ->  J.-U. Pott  |  ['J.-U. Pott']
F. Xu  ->  F. Xu  |  ['F. Xu']
H. Jiang  ->  H. Jiang  |  ['H. Jiang']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
K. Doi  ->  K. Doi  |  ['K. Doi']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 85 new papers today
          10 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/10 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2509.12316


extracting tarball to tmp_2509.12316...

 done.


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']


...

M. Benisty  ->  M. Benisty  |  ['M. Benisty']
Retrieving document from  https://arxiv.org/e-print/2509.13163


/tmp/ipykernel_3379/2822249172.py:52: LatexWarning: 2509.13122 did not run properly
bad escape \e at position 32
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


extracting tarball to tmp_2509.13163...

 done.


Found 78 bibliographic references in tmp_2509.13163/main.bbl.
Issues with the citations
syntax error in line 242: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.12316-b31b1b.svg)](https://arxiv.org/abs/2509.12316) | **Building up JWST-SUSPENSE: inside-out quenching at cosmic noon from age, Fe-, and Mg-abundance gradients**  |
|| C. M. Cheng, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2025-09-17*|
|*Comments*| *13 pages, 6 figures (excluding appendices); submitted to A&A*|
|**Abstract**|            Spatially resolved stellar populations of massive, quiescent galaxies at cosmic noon provide powerful insights into star-formation quenching and stellar mass assembly mechanisms. Previous photometric work has revealed that the cores of these galaxies are redder than their outskirts. However, spectroscopy is needed to break the age-metallicity degeneracy and uncover the driver of these colour gradients. Here, we derive age and elemental abundance gradients for 8 distant ($1.2 \lesssim z \lesssim 2.2$), massive ($10.3\lesssim\log({\rm M}_*/{\rm M}_\odot)\lesssim 11.1$), quiescent galaxies, by fitting full-spectrum models to ultra-deep NIRSpec-MSA spectroscopy from the JWST-SUSPENSE survey. We find that these galaxies have negative age, positive [Mg/H] and [Mg/Fe], and flat [Fe/H] gradients, implying that galaxy cores are older and Mg-deficient compared to galaxy outskirts. The age gradients indicate inside-out quenching, while the Mg-deficient cores suggest rapid gas expulsion as the central quenching mechanism. Thus, galaxy cores formed faster and quenched more efficiently than their outskirts. In this scenario, however, our [Fe/H] and [Mg/Fe] gradients are still puzzling. Our results contrast lower-redshift studies, which find flat age and [Mg/Fe] gradients and negative metallicity gradients. Additionally, we find a positive trend between age gradients and rotational support, and marginal trends between gradients and galaxy velocity dispersions and ages. We discuss our findings in the context of galaxy growth scenarios, including minor mergers and progenitor bias, and the possible occurrence of different quenching mechanisms across redshift. With this work, we present the first stellar population gradients from NIRSpec-MSA spectroscopy, in the largest current sample of distant, quiescent galaxies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.12479-b31b1b.svg)](https://arxiv.org/abs/2509.12479) | **BOWIE-ALIGN: Weak spectral features in KELT-7b's JWST NIRSpec/G395H transmission spectrum imply a high cloud deck or a low-metallicity atmosphere**  |
|| <mark>E.-M. Ahrer</mark>, et al. -- incl., <mark>P. Mollière</mark> |
|*Appeared on*| *2025-09-17*|
|*Comments*| *22 pages, 16 main figures, 7 tables, accepted for publication in MNRAS*|
|**Abstract**|            Hot Jupiters and their atmospheres are prime targets for transmission spectroscopy due to their extended atmospheres and the corresponding large signal-to-noise, providing the best possible constraints for the atmospheric carbon-to-oxygen (C/O) ratio and metallicity of exoplanets. Within BOWIE-ALIGN, we aim to compare JWST spectra of a sample of orbitally aligned and misaligned hot Jupiters orbiting F-type stars to probe the link between hot Jupiter atmospheres and planet formation history. Here, we present a near-infrared transmission spectrum of the aligned planet KELT-7b using one transit observed with JWST NIRSpec/G395H. We find weak features, only tentative evidence for H$_2$O and CO$_2$ in the atmosphere of KELT-7b. This poses a challenge to constrain the atmospheric properties of KELT-7b and two possible scenarios emerge from equilibrium chemistry and free chemistry retrievals: a high-altitude cloud deck muting all features or an extremely low metallicity atmosphere, respectively. The retrieved C/O ratios from our data reductions range from $0.43 - 0.74$, while the atmospheric metallicity is suggested to be solar to super-solar ($1-16 \times$ solar). Although these wide constraints prevent detailed conclusions about KELT-7b's formation history, a solar-to-super-solar metallicity would imply the accretion of solid material during its formation, which is valuable information for the survey's wider goals of understanding the relative importance of gaseous to solid accretion.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.12488-b31b1b.svg)](https://arxiv.org/abs/2509.12488) | **A high fraction of close massive binary stars at low metallicity**  |
|| H. Sana, et al. -- incl., <mark>S. Shahaf</mark> |
|*Appeared on*| *2025-09-17*|
|*Comments*| *32 pages, 12 figures*|
|**Abstract**|            At high metallicity, a majority of massive stars have at least one close stellar companion. The evolution of such binaries is subject to strong interaction processes, heavily impacting the characteristics of their life-ending supernova and compact remnants. For the low-metallicity environments of high-redshift galaxies constraints on the multiplicity properties of massive stars over the separation range leading to binary interaction are crucially missing. Here we show that the presence of massive stars in close binaries is ubiquitous, even at low metallicity. Using the Very Large Telescope, we obtained multi-epoch radial velocity measurements of a representative sample of 139 massive O-type stars across the Small Magellanic Cloud, which has a metal content of about one fifth of the solar value. We find that 45% of them show radial velocity variations which demonstrate that they are members of close binary systems, and predominantly have orbital periods shorter than one year. Correcting for observational biases indicates that at least 70[+11:-6]% of the O stars in our sample are in close binaries, and that at least 68[+7:-8]% of all O stars interact with a companion star during their lifetime. We found no evidence supporting a statistically significant trend of the multiplicity properties with metallicity. Our results indicate that multiplicity and binary interactions govern the evolution of massive stars and determine their cosmic feedback and explosive fates.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.13163-b31b1b.svg)](https://arxiv.org/abs/2509.13163) | **Euclid preparation. Using mock Low Surface Brightness dwarf galaxies to probe Wide Survey detection capabilities**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-09-17*|
|*Comments*| *21 pages, 18 figures, 7 tables*|
|**Abstract**|            Local Universe dwarf galaxies are both cosmological and mass assembly probes. Deep surveys have enabled the study of these objects down to the low surface brightness (LSB) regime. In this paper, we estimate Euclid's dwarf detection capabilities as well as limits of its MERge processing function (MER pipeline), responsible for producing the stacked mosaics and final catalogues. To do this, we inject mock dwarf galaxies in a real Euclid Wide Survey (EWS) field in the VIS band and compare the input catalogue to the final MER catalogue. The mock dwarf galaxies are generated with simple Sérsic models and structural parameters extracted from observed dwarf galaxy property catalogues. To characterize the detected dwarfs, we use the mean surface brightness inside the effective radius SBe (in mag arcsec-2). The final MER catalogues achieve completenesses of 91 % for SBe in [21, 24], and 54 % for SBe in [24, 28]. These numbers do not take into account possible contaminants, including confusion with background galaxies at the location of the dwarfs. After taking into account those effects, they become respectively 86 % and 38 %. The MER pipeline performs a final local background subtraction with small mesh size, leading to a flux loss for galaxies with Re > 10". By using the final MER mosaics and reinjecting this local background, we obtain an image in which we recover reliable photometric properties for objects under the arcminute scale. This background-reinjected product is thus suitable for the study of Local Universe dwarf galaxies. Euclid's data reduction pipeline serves as a test bed for other deep surveys, particularly regarding background subtraction methods, a key issue in LSB science.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.12978-b31b1b.svg)](https://arxiv.org/abs/2509.12978) | **Magnetic Reconnection as a Potential Driver of X-ray Variability in Active Galactic Nuclei**  |
|| C.-R. Hu, et al. -- incl., <mark>F. Xu</mark> |
|*Appeared on*| *2025-09-17*|
|*Comments*| *ApJ in press, 17 pages, 5 figures; DOI: https://doi.org/10.3847/1538-4357/adfed2*|
|**Abstract**|            We present a systematic analysis on the X-ray variability in 13 bright quasars at z > 4.5, combining recent Swift observations from 2021 to 2023 and archival multi-epoch observations. Upper limits of the luminosity measurements were included in the analysis by using the Kaplan-Meier estimator method. It is found that the high-z quasars exhibit X-ray variability on both short-term (hours-to-days) and intermediate-term (weeks-to-months) timescales, with short-term variability dominating the overall variation. A linear correlation exists between the global mean ($\mu_{\mathrm{L_{2-10\,keV}}}$) and standard deviation ($\sigma_{\mathrm{L_{2-10\,keV}}}$) of X-ray luminosities, which is independent of the X-ray photon index and optical-to-X-ray spectral slope. The localized stochastic magnetic reconnection mechanism is strongly favored, which can naturally lead to a scale-invariant power-law energy distribution and satisfactorily explain the correlation. The $\sigma$-$\mu$ correlation parallels with the well-documented rms-flux relation of low-z active galactic nuclei (AGNs), implying the magnetic reconnection mechanism could drive short-timescale X-ray variability in both high- and low-z AGNs. The highest-z quasar in our sample, J142952+544717 (z = 6.18), shows a luminosity distribution extending to ${10}^{47}\ \rm{erg\ {s}^{-1}}$ with a not conspicuous median luminosity. On the other hand, J143023+420436 (z = 4.7), which hosts the most relativistic jet among known high-z blazars, is dominated in the high-luminosity regime (${10}^{47}\ \rm{erg\ {s}^{-1}}$ ), making it an ideal target for multi-wavelength follow-up observations. J090630+693030 is found to have a rest-frame period of 182.46 days and J143023+420436 has a period of 16.89 days, both could be explained by the global evolution of plasmoid chains, in which magnetic islands formed during reconnection may merge successively.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.13122-b31b1b.svg)](https://arxiv.org/abs/2509.13122) | **Puffed-up Inner Rings and Razor-thin Outer Rings in Structured Protoplanetary Disks**  |
|| <mark>H. Jiang</mark>, et al. -- incl., <mark>M. Benisty</mark>, <mark>K. Doi</mark> |
|*Appeared on*| *2025-09-17*|
|*Comments*| *11 pages, 4 figures, 3 tables in the main text, a summary plot can be found in Fig. 4, Accepted for publication in ApJ*|
|**Abstract**|            The vertical distribution of pebbles in protoplanetary disks is a fundamental property influencing planet formation, from dust aggregation to the assembly of planetary cores. In the outer region of protoplanetary disks, the intensity of the optically thin but geometrically thick dust ring decreases along the minor axis due to reduced line-of-sight optical depth. Multi-ring disks thus provide an excellent opportunity to study the radial variation of the vertical properties of dust. We investigate the vertical dust distribution in 6 protoplanetary disks with resolved double rings, using high-resolution ALMA Band 6 continuum observations. By modeling the azimuthal intensity variations in these rings, we constrain the dust scale heights for each ring. Our results reveal a dichotomy: inner rings exhibit puffed-up dust layers with heights comparable to the gas scale height, while outer rings are significantly more settled, with dust scale heights less than 20\% of the gas scale height. This suggests a radial dependence in dust settling efficiency within the disks, potentially driven by localized planetary interactions or the global radial dependence of the Vertical Shear Instability (VSI). We discuss the implications of these findings for dust trapping, planet formation, and protoplanetary disk evolution. Our work highlights the importance of vertical dust distribution in understanding the early stages of planet formation and suggests that outer ($>80$~au), settled rings are preferred sites for planet formation over inner ($<80$~au), turbulent rings.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \e at position 32</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.12767-b31b1b.svg)](https://arxiv.org/abs/2509.12767) | **Simulation of Wavefront-based Disturbance Observers for Large Telescopes**  |
|| P. Jaufmann, et al. -- incl., <mark>J.-U. Pott</mark> |
|*Appeared on*| *2025-09-17*|
|*Comments*| *14 pages, 6 figures, SPIE Astronomical Telescopes + Instrumentation 2024 (Yokohama, Japan), Adaptive Optics Systems IX*|
|**Abstract**|            The performance of future observatories such as the Extremely Large Telescope is mainly limited by atmospheric turbulence and structural vibrations of the optical assembly. To further enhance the mitigation performance of adaptive optics, real-time information about the disturbances acting on the control loop is needed. Current systems therefore employ a combination of wavefront sensor- and accelerometer-based filters. In this work, methods using only data from natural- and laser guide star (NGS, LGS) measurements are presented, as telescopes like the Very Large Telescope already have multiple fast and high-resolution wavefront sensors installed. This approach also avoids the costly installation and operation of additional accelerometers on the optical elements. We introduce two innovative disturbance observer schemes to sense both turbulence and vibration information. A multi-rate estimator for atmospheric influences is based on Kalman filter theory and can incorporate NGS and LGS signals at different loop rates. The estimator for structural perturbations uses Gaussian process regression and can be implemented in an offline and online configuration. We validate the filter designs with data from a realistic end-to-end adaptive optics model with randomly generated turbulence and vibrations. The simulation is fed with on-sky data from the Adaptive Optics Facility of the Very Large Telescope. The presented disturbance observer schemes demonstrate promising results and may be considered as potential alternatives or extensions to existing techniques such as linear-quadratic controllers with Kalman filtering (LQG).         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.12775-b31b1b.svg)](https://arxiv.org/abs/2509.12775) | **Modal Identification of Mirror Vibrations at the VLT using Accelerometer Data**  |
|| P. Jaufmann, A. Buck, <mark>J.-U. Pott</mark>, O. Sawodny |
|*Appeared on*| *2025-09-17*|
|*Comments*| *6 pages, 7 figures, IEEE International Conference on Automation Science and Engineering (CASE), Los Angeles, USA*|
|**Abstract**|            Recent advances in ground-based astronomy have made it possible to create optical telescopes with primary mirrors up to 40 m in size. With growing mirror diameter, the suppression of non-atmospheric disturbances becomes increasingly important. Precise knowledge of the movement of telescope mirrors is essential for understanding and compensating for vibration-based perturbations. A model from VLT accelerometer data for each individual mirror is developed, while the influence of wind buffeting is accounted for by a von Karman wind model. To describe the relevant rigid body motion, we consider the piston, tip and tilt modes of the mirrors. The identification is validated by comparing the power spectral density of the measured and identified modes. Additionally, we assess the robustness of the approach by calculating the identification error over different sections of the data. The study indicates that the employed methods are adequate for the identification of modal telescope vibrations. It is anticipated that said findings will serve as a significant foundation for the development of advanced model-based AO controllers for large telescopes, such as linear quadratic Gaussian control.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.12551-b31b1b.svg)](https://arxiv.org/abs/2509.12551) | **Who Uses Whose Telescopes? Analyzing the Knowledge Geography and Research Dominance of Global Astronomical Facilities**  |
|| Y. Chen, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-09-17*|
|*Comments*| *45 pages, 12 figures*|
|**Abstract**|            Large-scale research infrastructures (LSRIs) are central to contemporary science policy, combining massive capital investments with international access regimes. Yet whether open access to these infrastructures translates into more equitable scientific authority remains contested. Astronomy provides a critical case: world-leading observatories are globally shared but embedded in specific national contexts. We compile a novel country--year dataset (1955--2025) linking the location of astronomical facilities with publication usage and authorship roles. This enables us to distinguish between hosting, using, and leading in telescope-based research. Our analysis reveals: (i) usage and impact are heavily concentrated in a small number of facility hubs; (ii) scientific leadership is even more unequal than access or usage (Gini coefficient 0.91 for first/corresponding authorship versus 0.85 for facilities and usage); (iii) hosting and leadership often decouple--countries such as Chile and South Africa mediate large publication volumes without commensurate gains in leading roles; and (iv) global leadership has shifted from U.S. dominance to a multi-hub system centered in the United States, Western Europe, China, Japan, and Australia. These findings challenge the assumption that international access alone democratizes science. We argue that converting participation into leadership requires domestic PI programs, investments in instrumentation and data pipelines, and governance models that distribute credit more equitably. The study highlights how the governance of LSRIs shapes global scientific hierarchies and offers design principles for infrastructures that seek not only to share data but also to broaden scientific authority.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2509.12316/./figures/grads_vs_params.png', 'tmp_2509.12316/./figures/params_vs_radius.png', 'tmp_2509.12316/./figures/image_cutout_overview.png']
copying  tmp_2509.12316/./figures/grads_vs_params.png to _build/html/
copying  tmp_2509.12316/./figures/params_vs_radius.png to _build/html/
copying  tmp_2509.12316/./figures/image_cutout_overview.png to _build/html/
exported in  _build/html/2509.12316.md
    + _build/html/tmp_2509.12316/./figures/grads_vs_params.png
    + _build/html/tmp_2509.12316/./figures/params_vs_radius.png
    + _build/html/tmp_2509.12316/./figures/image_cutout_overview.png
found figures ['tmp_2509.12479/./figures/all_reductions_lightcurves_with_residuals.png', 'tmp_2509.12479/./figures/POSEIDON_HST_free_chemistry_comparison_Hminus_NIRSpec_only_updated.png', 'tmp_2509.12479/./figures/nemesispy_equilibrium_chemistry_overplot.png']
copying  tmp_2509.12479/./figures/all_reductions_lightcurves_with_residuals.png to _build/html/
copying  tmp_2509.12479/.

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Building up _JWST_-SUSPENSE: inside-out quenching at cosmic noon from age, Fe-, and Mg-abundance gradients

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.12316-b31b1b.svg)](https://arxiv.org/abs/2509.12316)<mark>Appeared on: 2025-09-17</mark> -  _13 pages, 6 figures (excluding appendices); submitted to A&A_

</div>
<div id="authors">

C. M. Cheng, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** Spatially resolved stellar populations of massive, quiescent galaxies at cosmic noon provide powerful insights into star-formation quenching and stellar mass assembly mechanisms.  Previous photometric work has revealed that the cores of these galaxies are redder than their outskirts.  However, spectroscopy is needed to break the age-metallicity degeneracy and uncover the driver of these colour gradients.  Here, we derive age and elemental abundance gradients for 8 distant ( $1.2 \lesssim z \lesssim 2.2$ ), massive ( $10.3\lesssim\log({\rm M}_*/{\rm M}_\odot)\lesssim 11.1$ ), quiescent galaxies, by fitting full-spectrum models to ultra-deep NIRSpec-MSA spectroscopy from the _JWST_ -SUSPENSE survey.  We find that these galaxies have negative age, positive [ Mg/H ] and [ Mg/Fe ] , and flat [ Fe/H ] gradients, implying that galaxy cores are older and Mg-deficient compared to galaxy outskirts.  The age gradients indicate inside-out quenching, while the Mg-deficient cores suggest rapid gas expulsion as the central quenching mechanism.  Thus, galaxy cores formed faster and quenched more efficiently than their outskirts.  In this scenario, however, our [ Fe/H ] and [ Mg/Fe ] gradients are still puzzling.  Our results contrast lower-redshift studies, which find flat age and [ Mg/Fe ] gradients and negative metallicity gradients.  Additionally, we find a positive trend between age gradients and rotational support, and marginal trends between gradients and galaxy velocity dispersions and ages.  We discuss our findings in the context of galaxy growth scenarios, including minor mergers and progenitor bias, and the possible occurrence of different quenching mechanisms across redshift.  With this work, we present the first stellar population gradients from NIRSpec-MSA spectroscopy, in the largest current sample of distant, quiescent galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2509.12316/./figures/grads_vs_params.png" alt="Fig5" width="100%"/>

**Figure 5. -** Spatially resolved age and [Fe/H] gradients, normalized by $R_{\rm e}$(see Section \ref{sec:radii}), as a function of galaxy parameters.  We show the age gradients in the top row and the [Fe/H] gradients in the bottom row.  The outlying object with a strongly negative age gradient is 127154.  We show the integrated velocity dispersion ($\sigma$, from our alf$\alpha$ fits to the integrated spectra) in the left column, the integrated age (from our integrated alf$\alpha$ fits) in the middle, and $V_{r_{\rm e}}/\sigma_0$(measured in \citealt{Slob_2025}) on the right.  Horizontal dashed lines indicate where flat gradients would lie.  We perform a linear fit to the points in each panel, shown by the solid lines, with $1\sigma$ bootstrapped uncertainties indicated by the shaded regions. (*fig:grads_vs_params*)

</div>
<div id="div_fig2">

<img src="tmp_2509.12316/./figures/params_vs_radius.png" alt="Fig1" width="100%"/>

**Figure 1. -** Spatially-resolved stellar population parameters derived from our alf$\alpha$ full spectrum fits.  In the left column, we plot our measured parameters as a function of de-projected radius in units of $R_{\rm e}$(determined as described in Section \ref{sec:radii}).  In the right column, we plot our measured parameters as a function of de-projected radius in units of kpc.  We show age in the top row, [Fe/H] in the second row, [Mg/Fe] in the third row, and [Mg/H] in the bottom row.  In each panel, we show the measured parameters for each individual galaxy as points connected by lines.  We show the median parameters as stars connected by lines. (*fig:params_vs_radius*)

</div>
<div id="div_fig3">

<img src="tmp_2509.12316/./figures/image_cutout_overview.png" alt="Fig2" width="100%"/>

**Figure 2. -** Colour image cutouts of the 8 distant, quiescent galaxies in our sample.  Seven galaxies have NIRCam imaging, which we combine into RGB images here.  Galaxy 130040 does not have NIRCam imaging, so we show a cutout of the COSMOS _HST_/ACS F814W image (\citealt{COSMOS_ACS_mosaics, Scoville_2007}, note that this image is rotated by 90 degrees compared to the NIRCam images).  We highlight the regions that we combine for the core and outskirt spectra in blue and pink, respectively (see Section \ref{sec:extraction}).  We also indicate where shutter bars lie within these core and outskirt regions with shaded rectangles.  We indicate $1 R_{\rm e}$ in convolved space for each galaxy with an ellipse (see Section \ref{sec:radii}). (*fig:image_cutout_overview*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.12316"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Rsun}{R_{\odot}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\Rjup}{R_{\mathrm{J}}}$
$\newcommand{\Mjup}{M_{\mathrm{J}}}$
$\newcommand{\Rp}{R_{\mathrm{P}}}$
$\newcommand{\Mp}{M_{\mathrm{P}}}$
$\newcommand{\Rs}{R_{\mathrm{*}}}$
$\newcommand{\Ms}{R_{\mathrm{*}}}$
$\newcommand{\Teq}{T_{\mathrm{eq}}}$
$\newcommand{\Teff}{T_{\mathrm{eff}}}$
$\newcommand{◦ee}{^{\circ}}$
$\newcommand{\obliquity}{|\lambda|}$
$\newcommand{\mystar}{KELT-7\xspace}$
$\newcommand{\myplanet}{KELT-7 b\xspace}$
$\newcommand{\eureka}{\texttt{Eureka!}\xspace}$
$\newcommand{\tiberius}{\texttt{Tiberius}\xspace}$
$\newcommand{\exoticjedi}{\texttt{ExoTiC-JEDI}\xspace}$
$\newcommand{\pRT}{\texttt{pRT}\xspace}$
$\newcommand{\poseidon}{\texttt{POSEIDON}\xspace}$
$\newcommand{\nemesispy}{\texttt{NEMESISPY}\xspace}$
$\newcommand{\nemesis}{\texttt{NEMESIS}\xspace}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# BOWIE-ALIGN: Weak spectral features in KELT-7b's JWST NIRSpec/G395H transmission spectrum imply a high cloud deck or a low-metallicity atmosphere

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.12479-b31b1b.svg)](https://arxiv.org/abs/2509.12479)<mark>Appeared on: 2025-09-17</mark> -  _22 pages, 16 main figures, 7 tables, accepted for publication in MNRAS_

</div>
<div id="authors">

<mark>E.-M. Ahrer</mark>, et al. -- incl., <mark>P. Mollière</mark>

</div>
<div id="abstract">

**Abstract:** Hot Jupiters and their atmospheres are prime targets for transmission spectroscopy due to their extended atmospheres and the corresponding large signal-to-noise, providing the best possible constraints for the atmospheric carbon-to-oxygen (C/O) ratio and metallicity of exoplanets.Within BOWIE-ALIGN, we aim to compare JWST spectra of a sample of orbitally aligned and misaligned hot Jupiters orbiting F-type stars to probe the link between hot Jupiter atmospheres and planet formation history.Here, we present a near-infrared transmission spectrum of the aligned planet KELT-7b using one transit observed with JWST NIRSpec/G395H. We find weak features, only tentative evidence for $\ce{H2O}$ and $\ce{CO2}$ in the atmosphere of KELT-7b. This poses a challenge to constrain the atmospheric properties of KELT-7b and two possible scenarios emerge from equilibrium chemistry and free chemistry retrievals: a high-altitude cloud deck muting all features or an extremely low metallicity atmosphere, respectively.The retrieved C/O ratios from our data reductions range from $0.43 - 0.74$ , while the atmospheric metallicity is suggested to be solar to super-solar ( $1-16 \times$ solar). Although these wide constraints prevent detailed conclusions about KELT-7b's formation history, a solar-to-super-solar metallicity would imply the accretion of solid material during its formation, which is valuable information for the survey's wider goals of understanding the relative importance of gaseous to solid accretion.

</div>

<div id="div_fig1">

<img src="tmp_2509.12479/./figures/all_reductions_lightcurves_with_residuals.png" alt="Fig8" width="100%"/>

**Figure 8. -** Extracted raw spectroscopic light curves (top row) and corresponding residuals (bottom row) of each reduction (columns; $\eureka$, $\exoticjedi$ and $\tiberius$) of $\myplanet$ with NIRSpec/G395H at a resolution of R=400. The white area corresponds to the gap between the two detectors (NRS1 and NRS2). (*fig:light-curves-2d*)

</div>
<div id="div_fig2">

<img src="tmp_2509.12479/./figures/POSEIDON_HST_free_chemistry_comparison_Hminus_NIRSpec_only_updated.png" alt="Fig11" width="100%"/>

**Figure 11. -** Top: $\poseidon$ free chemistry retrievals comparison between using only JWST data (orange) and in combination with previously published HST data (pink: excluding \ce{H-}; purple: including \ce{H-}). For clarity, we include the WFC/G141 data twice, as the difference in offset retrieved when including \ce{H-} is significant. The colour of the data corresponds to the offset applied from each model. For the UVIS/G280 data we only applied one offset as the uncertainties are relatively large and the offset difference is not as significant. \newline Bottom: The posterior plots for the retrieved species, cloud deck and temperature ($T$), as well as scattering slope parameters for the retrievals with HST data ($\log a, \gamma$). The colours correspond to the individual models, demonstrating that the inclusion of HST data does not provide further constraints for the cloud deck or abundances as the unknown \ce{H-} abundance in combination with the lack of overlapping wavelengths prevents inferences about the continuum of $\myplanet$'s atmosphere. Quoted median values and 1$\sigma$ errors correspond to the posteriors retrieved on the JWST and HST data when including \ce{H-} opacity, shown with the vertical solid and dashed lines. (*fig:appendix_g395h_hst*)

</div>
<div id="div_fig3">

<img src="tmp_2509.12479/./figures/nemesispy_equilibrium_chemistry_overplot.png" alt="Fig16" width="100%"/>

**Figure 16. -** Top: NEMESISPY equilibrium chemistry retrievals comparison between using JWST data in combination with previously published HST data (purple: including \ce{H-} and \ce{e-}) and JWST data only (orange). The colour of the data corresponds to the offset applied from each model. Whilst the retrieval including the HST data was conducted with the G280 dataset held fixed, here we shift the plot such that the offsets are shown relative to NRS1 to facilitate comparison with figure \ref{fig:appendix_g395h_hst}. \newline Bottom: The posterior plots for the metallicity and C:O ratio, H$^-$ and e$^-$, R$_{\mathrm{ref}}$ and P$_{\mathrm{ref}}$; temperature (P1,P2,P2,T0,alpha1,alpha2); cloud properties (cloud top pressure, haze scattering index, haze opacity and cloud fraction); offsets; and stellar parameters (heterogeneity fraction, photospheric temperature, heterogeneity temperature relative to photosphere).  The colours correspond to the individual models, demonstrating that the inclusion of HST data does not provide further constraints for the cloud deck or metallicity. The discrepancy in the retrieved radius is due to the two retrievals referencing the instrument offsets to different baseline instruments (G280 for the full retrieval and NRS1 for the JWST-only retrieval). The offsets for both retrievals are presented relative to NRS1 to aid comparison. (*fig:appendix_g395h_hst_nemesispy*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.12479"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\bibinfo}[2]{#2}$
$\newcommand{\eprint}[2][]{\url{#2}}$
$\newcommand{\doi}[1]{\url{https://doi.org/#1}}$
$\newcommand{\giraffe}{{\sc giraffe}}$
$\newcommand{\figurename}{Figure}$
$\newcommand{\figurename}{Extended Data Figure}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\cms}{cm s^{-2}}$
$\newcommand{\ergscm}{erg s^{-1} cm^{-2}}$
$\newcommand{\ergs}{erg s^{-1}}$
$\newcommand{\zsun}{Z_{\odot}}$
$\newcommand{\Zsun}{Z_{\odot}}$
$\newcommand{\lsol}{L_{\odot}}$
$\newcommand{\lsun}{L_{\odot}}$
$\newcommand{\Lsun}{L_{\odot}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\msunyr}{M_{\odot} yr^{-1}}$
$\newcommand{\msol}{M_{\odot}}$
$\newcommand{\rsol}{R_{\odot}}$
$\newcommand{\rsun}{R_{\odot}}$
$\newcommand{\Rsun}{R_{\odot}}$
$\newcommand{\teff}{T_\mathrm{eff}}$
$\newcommand{\logg}{\log g}$
$\newcommand{\mdot}{M_\odot}$
$\newcommand{\vinf}{v_\infty}$
$\newcommand{\vsini}{\varv \sin i}$
$\newcommand{\ha}{H{\alpha}}$
$\newcommand{\halpha}{H{\alpha}}$
$\newcommand{\hb}{H{\beta}}$
$\newcommand{\hbeta}{H{\beta}}$
$\newcommand{\hdelta}{H{\delta}}$
$\newcommand{\hgamma}{H{\gamma}}$
$\newcommand{\l}{\lambda}$
$\newcommand{\hea}{\ion{He}{i}}$
$\newcommand{\heb}{\ion{He}{ii}}$
$\newcommand{\snr}{S/N}$
$\newcommand{\bloem}{BLOeM}$
$\newcommand{\flames}{\sc flames}$
$\newcommand{\aap}{Astron.~Astrophys.}$
$\newcommand{\aaps}{Astron.~Astrophys.~Suppl.}$
$\newcommand{\aapr}{Astron.~Astrophys.~Rev.}$
$\newcommand{\aj}{Astron.~J}$
$\newcommand{\apj}{Astrophys.~J}$
$\newcommand{\apjs}{Astrophys.~J.~Suppl.}$
$\newcommand{\apjl}{Astrophys.~Lett.}$
$\newcommand{\aplett}{Astrophys.~Lett.}$
$\newcommand{\araa}{Ann.~Rev. Astron.~Astroph.}$
$\newcommand{\mnras}{Mon.~Not.~Royal~Ac.~Soc.}$
$\newcommand{\nat}{Nature}$
$\newcommand{\pasp}{Publ. Astron. Soc. Pac.}$
$\newcommand{\pasa}{Publ. Astron. Soc. Australia}$
$\newcommand\url{#1}$
$\newcommand{\urlprefix}{URL }$</div>



<div id="title">

# A high fraction of close massive binary stars at low metallicity

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.12488-b31b1b.svg)](https://arxiv.org/abs/2509.12488)<mark>Appeared on: 2025-09-17</mark> -  _32 pages, 12 figures_

</div>
<div id="authors">

H. Sana, et al. -- incl., <mark>S. Shahaf</mark>

</div>
<div id="abstract">

**Abstract:** At high metallicity, a majority of massive stars have at least one close stellar companion.The evolution of such binaries is subject to strong interaction processes, heavily impacting the characteristics of their life-ending supernova and compact remnants.For the low-metallicity environments of high-redshift galaxies constraints on the multiplicity properties of massive stars over the separation range leading to binary interaction are crucially missing.  Here we show that the presence of massive stars in close binaries is ubiquitous, even at low metallicity.Using the Very Large Telescope, we obtained multi-epoch radial velocity measurements of a representative sample of 139 massive O-type stars across the Small Magellanic Cloud, which has a metal content of about one fifth of the solar value. We find that 45 \% of them show radial velocity variations which demonstrate that they are members of close binary systems, and predominantly have orbital periods shorter than one year. Correcting for observational biases indicatesthat at least $70^{+11}_{-6}$ \% of the O stars in our sample are in close binaries, and that at least $68^{+7}_{-8}$ \% of all O stars interact with a companion star during their lifetime. We found no evidence supporting a statistically significant trend of the multiplicity properties with metallicity.Our results indicate that multiplicity and binary interactions govern the evolution of massive stars and determine their cosmic feedback and explosive fates.

</div>

<div id="div_fig1">

<img src="tmp_2509.12488/./Sana_Fig4.png" alt="Fig4" width="100%"/>

**Figure 4. -** Metallicity dependence of the intrinsic binary fraction of O-type stars (data: circles, model: green; this work) and of solar-mass stars (data: diamonds, model: red; \cite{Moe2019}).
The best-fit parameters and their standard deviations are computed with a linear regression to the displayed data points and their 1$\sigma$ error-bars. The green and red shaded regions show the 68\%(1$\sigma$) confidence intervals of the regressions to the high-mass and low-mass data, respectively. The slope for massive stars ($-0.11 \pm 0.15$, solid line) is consistent with a constant line (dashed line), but does not allow to reject a slight trend with metallicity at the level of the slope reported for solar-mass stars ($-0.16 \pm0.01$, dotted line).    (*f:reg*)

</div>
<div id="div_fig2">

<img src="tmp_2509.12488/./Sana_Fig1.png" alt="Fig1" width="100%"/>

**Figure 1. -** **Distribution of O-type stars** in the $\bloem$ sample overlaid on a VISTA Y-J-K$_\mathrm{S}$ false-color image of the SMC. Image credit:ESO/VISTA VMC. Large circles show the 8 fields of view of the $\bloem$ campaign. Diamonds indicate detected O-type spectroscopic binaries (SB); circles, RV constant, presumably single stars.  (*f:fov*)

</div>
<div id="div_fig3">

<img src="tmp_2509.12488/./Sana_Fig2.png" alt="Fig2" width="100%"/>

**Figure 2. -** {Binary detection probability} of the $\bloem$ survey for O stars in our sample as a function of the orbital period. The plain green line is computed while considering the Doppler shift of the primary star only (main method). The dashed, purple line also include the line-blending detection bias, which reduces detection at long periods. The dotted, red lines indicates the fraction of the simulated systems that display eclipses according to Equation \ref{eq:eclipses}. Detection probability curves as function of other parameters are provided in Extended Data Figure 2. (*f:detect_p*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.12488"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\AL}[1]{\textcolor{teal}{#1}}$
$\newcommand{\orcid}[1]$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# $\Euclid$ preparation. Using mock Low Surface Brightness dwarf galaxies to probe Wide Survey detection capabilities

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.13163-b31b1b.svg)](https://arxiv.org/abs/2509.13163)<mark>Appeared on: 2025-09-17</mark> -  _21 pages, 18 figures, 7 tables_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Local Universe dwarf galaxies are both cosmological and mass assembly probes. Deep surveys have enabled the study of these objects down to the low surface brightness (LSB) regime. In this paper, we estimate $\Euclid$ 's dwarf detection capabilities as well as limits of its MERge processing function (MER pipeline), responsible for producing the stacked mosaics and final catalogues. To do this, we inject mock dwarf galaxies in a real Euclid Wide Survey (EWS) field in the VIS band and compare the input catalogue to the final MER catalogue. The mock dwarf galaxies are generated with simple Sérsic models and structural parameters extracted from observed dwarf galaxy property catalogues. To characterize the detected dwarfs, we use the mean surface brightness inside the effective radius ${\rm SB}_{\rm e}$ (in mag arcsec $^{-2}$ ). The final MER catalogues achieve completenesses of $91 \%$ for ${\rm SB}_{\rm e}\in[21,24]$ , and $54 \%$ for ${\rm SB}_{\rm e}\in[24,28]$ . These numbers do not take into account possible contaminants, including confusion with background galaxies at the location of the dwarfs. After taking into account those effects, they become respectively $86 \%$ and $38 \%$ . The MER pipeline performs a final local background subtraction with small mesh size, leading to a flux loss for galaxies with $R_{\rm e}>10\arcsec$ . By using the final MER mosaics and reinjecting this local background, we obtain an image in which we recover reliable photometric properties for objects under the arcminute scale. This background-reinjected product is thus suitable for the study of Local Universe dwarf galaxies. $\Euclid$ 's data reduction pipeline serves as a test bed for other deep surveys, particularly regarding background subtraction methods, a key issue in LSB science.

</div>

<div id="div_fig1">

<img src="tmp_2509.13163/./figures/histogram_count_vs_SB_corrected.png" alt="Fig14.1" width="50%"/><img src="tmp_2509.13163/./figures/histogram_count_vs_Re_corrected.png" alt="Fig14.2" width="50%"/>

**Figure 14. -** Histograms of the input ${\rm SB}_{\rm e}$(left panel) and the input $R_{\rm e}$(right panel), colour-coded according to their detection by eye and in the MER catalogue, $N_\sfont{SEARCH}$ being the number of MER sources found by using the search radius $R_{\sfont{SEARCH}}$ for the crossmatch. It is worth noting that the dwarfs in MER catalogues and those with $N_\sfont{SEARCH}>1$ are also detected by eye. In Appendix \ref{AppA}, we also provide the histogram of $\langle \mu_\sfont{I} \rangle$ as defined in \cite{Q1-SP001}. In this plot, all the dwarfs (nucleated or not) are included. (*fig:detect_catalog_histo*)

</div>
<div id="div_fig2">

<img src="tmp_2509.13163/./figures/combined_non_nucleated.png" alt="Fig17" width="100%"/>

**Figure 17. -** Input $R_{\rm e}$ as a function of the input total magnitude in $\IE$(left panel) and the corresponding histogram of the input ${\rm SB}_{\rm e}$(right panel), colour-coded according to their detection or non-detection by eye in the three types of products: NOBG, VIS BGSUB, and the final MER mosaic subtracted by VIS and MER backgrounds. Only the non-nucleated dwarfs, at all distances, have been used in this analysis.  We labelled the number of dwarfs that are visually detected in each ${\rm SB}_{\rm e}$ bin. The dashed rectangle shows examples of objects that were detected differently with the various background treatments. (*fig:detection*)

</div>
<div id="div_fig3">

<img src="tmp_2509.13163/./figures/diff_mag_bgsub_combined.png" alt="Fig8" width="100%"/>

**Figure 8. -** Difference between the \texttt{SourceXtractor++} measured magnitude in the final MER catalogue and the input magnitude of injected dwarfs as a function of $R_{\rm e}$, for all dwarfs detected in the final MER catalogue. (*fig:detect_catalog_2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.13163"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

128  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
